### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 1. Orbit Algorithms: A Shorter History of CGT

##  Tree Traversal: BFS vs DFS

* Suppose we want to visit all the nodes of a (rooted) tree like this (with root node in white):

![e7 tree](images/e7tree.png)

* **Breath First Search** (BFS) and **Depth First Search**  (DFS) provide strategies for doing this in a simple and systematic fashion.
* For this, we assume that each **node** $x$ in the tree knows all its **children** as a list $x$.next.
* Then the two **algorithms** for visiting all nodes of a tree with root $x$ can be described as follows.

![bfs vs dfs](images/bfsdfs.png)

* In practice, BFS can take advantage of the way dynamic lists in a (e.g. GAP, Python, ...) `for` loop are largely organized as queues.

In [ ]:
BFS := function(x, visit)
    local   Q,  y,  z;
    Q:= [x];
    for y in Q do
        visit(y);
        for z in y.next do
            Add(Q, z);
        od;
    od;
end;

* While DFS, using the function call stack as its stack, can be implemented recursively. 

In [ ]:
DFS := function(x, visit)
    local   z;
    visit(x);
    for z in x.next do
        DFS(z, visit);
    od;
end;

* Applied to the above example ...

In [ ]:
nodes:= List([1..7], i-> rec(id:= i, next:= []));;
root:= nodes[6];;  parent := [3,4,4,5,6,0,6];;
for i in [1..Length(parent)] do
    if parent[i] > 0 then
        Add(nodes[parent[i]].next, nodes[i]);
    fi;
od;

* ... with a visitor that simply prints the names of each node it encounters ...

In [ ]:
print := function(x)  Print(x.id, ", ");  end;

* ... BFS and DFS list all the nodes in slightly different orders:

In [ ]:
BFS(root, print);

In [ ]:
DFS(root, print);

* Recall the graph: ![e7 tree](images/e7tree.png)

* the recursive strategy can be modified to suit a specific purpose, e.g., to print a tree as a tree 

In [ ]:
t_indent := 0;;  t_nl := false;;
t_print := function(x)
    local   c;
    if t_nl then
        Print(RepeatedString(" ", t_indent));  t_nl:= false;
    fi;
    Print("-", x.id);
    if Length(x.next) = 0 then
        Print("\n");  t_nl:= true;
    fi;
    t_indent:= t_indent + 2;
    for c in x.next do  t_print(c);  od;
    t_indent:= t_indent - 2;
end;

In [ ]:
t_print(root);

### Graph Traversal

* Both BFS and DFS can be applied to a (simple or directed) **graph**.
* The same node then can possibly be reached through different paths.
* Care needs to be taken to manage these repeat encounters.

###  Applications

* distance between nodes
* shortest paths
* connected components
* ...

## Orbits

* **Group actions** are a rich source of graphs.
* Here, the **nodes** of the graph are the elements $x$ of the domain that is acted upon.
* The **edges** of the graph are of the form $x \stackrel{s}{\longrightarrow} x.s$, implicitly given by the action.
* The elements $s$ typically come from a set of **generators** of the acting group.

<div class="alert alert-danger">

**Definition.**
Let $G$ be a group acting on a set $X$ via $(x, a) \mapsto x.a$, and suppose that $G = \langle S \rangle$ for 
some $S \subseteq G$. The **action graph** of $S$ and $X$ is the (directed) graph with 
* **vertices:** $x \in X$, and
* **edges:** $x \stackrel{s}{\longrightarrow} x.s$ for $x \in X$, $s \in S$.
</div>

* We formulate and apply a variant of BFS called **orbit algorithm** to compute properties of this action.

* In order to have some groups to play with, provide a list of transpositions of adjacent points.

In [ ]:
transpositions:= n -> List([2..n], j-> (j-1,j));

* For example, on $4$ points (which group do they generate?):

In [ ]:
swaps:= transpositions(4);

### Orbit Algorithm

* The plain orbit algorithm:  BFS with $x$.next ${} = \{x.s : s \in S\}$.
* The action of $G$ on $X$ is described by an **action function** `under`: calling `under(x, s)` returns $x.s$ (&ldquo;$x$ under $s$&rdquo;).
* Given: a list `aaa` of generating operators, a point `x` of the domain $X$, and an action function `under`, we compute the **orbit** $x^G = \{x.a : a \in G\}$ of the point `x` under the the action of the group generated by `aaa`.

In [ ]:
orbit:= function(aaa, x, under)
    local a, y, z, list;
    list:= [x];
    for y in list do
        for a in aaa do
            z:= under(y, a);
            if z <> y and not z in list then Add(list, z); fi;
        od;
    od;
    return list;
end;

To find the orbit of a point $x$ under the group generated by the swaps:
* apply the orbit algorithm to (i) the swaps, (ii) the point $x$, (iii) the standard action *on points* $(x, a) \mapsto x^a$

In [ ]:
onPoints:= function(x, a)  return x^a;  end;

In [ ]:
orbit(swaps, 2, onPoints);

* in GAP, this action is called `OnPoints`:

In [ ]:
orbit(swaps, 3, OnPoints);

### Elements

To find the **elements** of the group generated by the swaps:
* apply the orbit algorithm to (i) the swaps, (ii) the identity permutation, (iii) the action *on the right* $(x, a) \mapsto x a$

In [ ]:
onRight:= function(x, a)  return x * a;  end;

In [ ]:
orbit(swaps, (), onRight);

* in GAP, this action is called `OnRight`

In [ ]:
orbit(swaps, (), OnRight);

* In CGT, it is customary to represent a group $G = \langle A \rangle$ by a list $A$ of generators (avoiding the need to list all its elements where possible).

* Let's turn the list of generators into a group object, and from now on formulate algorithms in terms of the group. 

In [ ]:
swaps;
group:= GroupWithGenerators(swaps);
GeneratorsOfGroup(group);

In [ ]:
elements:= function(group)
    return orbit(GeneratorsOfGroup(group), Identity(group), OnRight);
end;

* test it

In [ ]:
eee:= elements(group);

In [ ]:
Length(eee);

### Words in the generators

* express these elements as words in the generators

In [ ]:
onWords := function(word, s)
    return Concatenation(word, [s]);
end;

In [ ]:
orbit_with_words:= function(aaa, x, under)
    local   list,  words,  i,  k,  z;
    list:= [x];  words:= [[]];  i:= 0;
    while i < Length(list) do
        i:= i+1;
        for k in [1..Length(aaa)] do
            z:= under(list[i], aaa[k]);
            if not z in list then
                Add(list, z);
                Add(words, onWords(words[i], k));
            fi;
        od;
    od;
    return rec(list:= list, words:= words);
end;

In [ ]:
www:= orbit_with_words(swaps, (), onRight).words;

## Conjugacy Classes

* With suitable arguments, `OnPoints` also implements the **conjugation** action of a group $G$ on itself.
* We can use the orbit algorithm to compute the conjugacy class of a single element in $G$, and to find the list of all conjugacy classes of $G$.

###  Conjugacy Class

* to find the conjugacy class of an element

In [ ]:
class:= function(group, x)
    return Set(orbit(GeneratorsOfGroup(group), x, OnPoints));
end;

* test on a random element

In [ ]:
a:= Random(eee);

In [ ]:
class(group, a);

* a conjugacy class should know its group.
* in GAP, the class of `a` in `G` is denoted (and constructed) as `a^G`.

In [ ]:
cl:= a^group;

* such a class has: **elements**, a **representative**, and an **acting domain** (the group `G`).

In [ ]:
Elements(cl);
Representative(cl);
ActingDomain(cl);

###  Conjugacy Classes

* to find all conjugacy classes, the list of generators itself needs to be closed under conjugation.
* to find the closure of a set under conjugation, we can use a version of the orbit algorithm that initializes its queue with several points, not just one.

In [ ]:
orbits:= function(aaa, xxx, under)
    local a, y, z, list;
    list:= ShallowCopy(xxx);
    for y in list do
        for a in aaa do
            z:= under(y, a);
            if z <> y and not z in list then Add(list, z); fi;
        od;
    od;
    return list;
end;

In [ ]:
orbits(swaps, swaps, onPoints);

In [ ]:
onClasses:= function(x, a)
    return OnRight(Representative(x), a)^ActingDomain(x);
end;

In [ ]:
conjugacyClasses:= function(group)
    local gens;
    gens:= GeneratorsOfGroup(group);
    return orbit(orbits(gens, gens, OnPoints), Identity(group)^group, onClasses);
end;

In [ ]:
cc:= conjugacyClasses(group);

In [ ]:
List(cc, Size);

## Subgroups

* The power set $2^G$ is a monoid wrt. to set union, generated by the singleton sets $\{a\}$, $a \in G$, as atoms.
* $2^G$ acts on the subgroups $H$ of $G$ via closure: $H.A = \langle H, A \rangle$.  (Check!)
* In fact, each subgroup lies in the orbit of the trivial subgroup of $G$.  (Check!)

In [ ]:
onGroups:= function(x, a)
    return ClosureGroup(x, a);
end;

In [ ]:
subgroups:= function(group)
    return orbit(Elements(group), TrivialSubgroup(group), onGroups);
end;

In [ ]:
Size(group);

In [ ]:
subs:= subgroups(group);

In [ ]:
Length(subs);

### Conjugacy Classes of Subgroups

* combining ideas from above ...

In [ ]:
onSubgroupClasses:= function(x, a)
    return onGroups(Representative(x), a)^ActingDomain(x);
end;

* also it suffices to consider **zuppos** as potential generators ...

In [ ]:
subgroupClasses:= function(group)
    return orbit(Zuppos(group), TrivialSubgroup(group)^group, onSubgroupClasses);
end;

In [ ]:
ccs:= subgroupClasses(group);

In [ ]:
Length(ccs);

In [ ]:
List(ccs, Size);

In [ ]:
Sum(ccs, Size);

In [ ]:
ccs:= subgroupClasses(GroupWithGenerators(transpositions(5)));;
Length(ccs);

## Stabilizer and Transversal

* we can use a variant of the orbit algorithm to determine (and remember), for each point $y$ in the $G$-orbit of $x$, a representative element $a \in G$ with $x.a = y$.
* this list of representatives will form a **transversal** of the cosets of the stabilizer of $x$ in $G$.
* by Schreier's Theorem, a generating set for the **stabilizer** can be computed from the transversal.

### Transversal

In [ ]:
orbit_with_transversal:= function(aaa, x, under)
    local   list,  reps,  i,  k,  z;
    list:= [x];  reps:= [aaa[1]^0];  i:= 0;
    while i < Length(list) do
        i:= i+1;
        for k in [1..Length(aaa)] do
            z:= under(list[i], aaa[k]);
            if not z in list then
                Add(list, z);
                Add(reps, reps[i] * aaa[k]);
            fi;
        od;
    od;
    return rec(list:= list, reps:= reps);
end;

In [ ]:
swaps:= transpositions(5);;
transversal:= orbit_with_transversal(swaps, 2, onPoints);

### Stabilizer

<div class="alert alert-danger">

**Schreier's Theorem.**
Suppose a group $G$, generated by a set $S$,
acts on a set $X$ and that $\{f_y : y \in x^G\}$ is a transversal
of the orbit of $x \in X$.  Then
$$
    \{f_y a f_{y.a}^{-1}: a \in A,\, y \in x^G\}
$$
is a set of generators for the stabilizer $G_x$ of $x$ in $G$.
    
</div>

In [ ]:
orbit_with_stabilizer:= function(aaa, x, under)
    local   list,  reps,  stab,  i,  k,  l,  z;
    list:= [x];  reps:= [aaa[1]^0];  stab:= [];  i:= 0;
    while i < Length(list) do
        i:= i+1;
        for k in [1..Length(aaa)] do
            z:= under(list[i], aaa[k]);
            l:= Position(list, z);
            if l = fail then
                Add(list, z);
                Add(reps, reps[i] * aaa[k]);
            else   # x^(reps[i] * a) = x^reps[l]
                Add(stab, reps[i] * aaa[k] / reps[l]);
            fi;
        od;
    od;
    return rec(list:= list, reps:= reps, stab:= stab);
end;

In [ ]:
stabilizer:= orbit_with_stabilizer(swaps, 2, OnPoints);

In [ ]:
Difference(stabilizer.stab, [()]);

## Stabilizer Chain

* Applying these ideas along a chain of stabilizers can yield information about the whole group.

### Order of the Group

* Orbit-Stabilzer Lemma: $|G| = |x^G| \, |G_x| = |x^G| \, |y^{G_x}| \, |G_{x, y}| = {\dots}$

In [ ]:
sizeOfGroup:= function(group)
    local x, orb, stab;
    if group = TrivialSubgroup(group) then return 1; fi;
    x:= LargestMovedPoint(group);
    orb:= orbit_with_stabilizer(GeneratorsOfGroup(group), x, OnPoints);
    stab:= Subgroup(group, Difference(orb.stab, [()]));
    return sizeOfGroup(stab) * Length(orb.reps);
end;

In [ ]:
sizeOfGroup(group);

In [ ]:
group:= Group(transpositions(10));

In [ ]:
sizeOfGroup(group);

In [ ]:
m24:= Group([
  (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23),
  (3,17,10,7,9)(4,13,14,19,5)(8,18,11,12,23)(15,20,22,21,16),
  (1,24)(2,23)(3,12)(4,16)(5,18)(6,10)(7,20)(8,14)(9,21)(11,17)(13,22)(15,19) 
]);;
sizeOfGroup(m24);

###  Random Element

* random elements: along the same lines as `sizeOfGroup`

In [ ]:
randomGroupElement:= function(group)
    local x, orb, stab;
    if group = TrivialSubgroup(group) then return Identity(group); fi;
    x:= LargestMovedPoint(group);
    orb:= orbit_with_stabilizer(GeneratorsOfGroup(group), x, OnPoints);
    stab:= Subgroup(group, Difference(orb.stab, [()]));
    return randomGroupElement(stab) * Random(orb.reps);
end;

In [ ]:
randomGroupElement(m24);

* Check: These random elements are **uniformly distributed**!

In [ ]:
group:= Group(transpositions(4));
Collected(List([1..2400], i-> randomGroupElement(group)));

Also, using similar ideas:
* **membership** test $a \in G$?
* express element as **word in the generators**: $a = s_1 \dotsm s_k$
* **homomorphisms** (defined on generators): $\phi(a) = \phi(s_1) \dotsm \phi(s_k)$

## Graphs

* With a further small modification, the orbit algorithm can keep track of edges and thus construct the **action graph**.  
* Recall that this is a labelled directed graph, with vertices $x, y \in X$ and edges $x \stackrel{a}{\to} y$, whenever $x.s = y$ for $x, y \in X$, and $s \in S$.  
* However, we need to decide on a data structure for such graphs first.  The simplest, perhaps, is a list of pairs of indices, each representing an edge.

In [ ]:
LoadPackage("jupyterviz");  # thanks: Nathan Carter @bentley.edu
opts:= rec(vertexwidth:= 12, vertexheight:= 12, edgecolor:= "#def");

In [ ]:
PlotGraph([[2,4],[2,6],[2,8],[4,4]], opts);

In [ ]:
edges:= List([1..Length(parent)], i -> [i, parent[i]]);
edges[6][2]:= 6;
PlotGraph(edges, opts);

In [ ]:
orbit_with_edges:= function(aaa, x, under)
    local   list,  edges,  i,  k,  l,  z;
    list:= [x];  edges:= [];  i:= 0;
    while i < Length(list) do
        i:= i+1;
        for k in [1..Length(aaa)] do
            z:= under(list[i], aaa[k]);
            l:= Position(list, z);
            if l = fail then
                Add(list, z);
                l := Length(list);
            fi;
            Add(edges, [i, l]);
        od;
    od;
    return rec(list:= list, edges:= edges);
end;

In [ ]:
edges:= orbit_with_edges(swaps, 2, OnPoints).edges;

In [ ]:
PlotGraph(Set(edges), opts);

In [ ]:
edges:= orbit_with_edges(swaps, [1,2], OnPairs).edges;;
edges:= Filtered(Set(edges), x-> x[1] <> x[2]);

In [ ]:
PlotGraph(edges, opts);

In [ ]:
edges:= orbit_with_edges(transpositions(6), [1,2], OnSets).edges;;
edges:= Filtered(Set(edges), x-> x[1] <> x[2]);

In [ ]:
PlotGraph(Set(edges), opts);

In [ ]:
edges:= orbit_with_edges(transpositions(4), (), OnRight).edges;;
edges:= Filtered(Set(edges), x-> x[1] <> x[2]);

In [ ]:
PlotGraph(edges, opts);

### Permutations

* The action graph is in fact a directed graph.
* It encodes the permutations induced by the action on the domain.
* For this (and other applications) it is convenient to store the edge information in a different format: as image lists.

In [ ]:
orbit_with_images := function(aaa, x, under)
    local   list,  images,  i,  k,  l,  z;
    list := [x];  i := 0;
    images := List(aaa, x -> []);
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                l := Length(list);
            fi;
            images[k][i] := l;
        od;
    od;
    return rec(list := list, images := images);
end;

In [ ]:
swaps := transpositions(5);
orb := orbit_with_images(transpositions(5), 1, OnPoints);

In [ ]:
List(orb.images, PermList);

In [ ]:
List(orbit_with_images(swaps, [1,2], OnSets).images, PermList);

### Rearrangements

* Example Polya Action: rearrangements of a word.
* In GAP, the action on the positions of a list is called `Permuted`.

In [ ]:
word := [1,1,2,2,2];
orb := orbit_with_images(swaps, word, Permuted);;
orb.list;

In [ ]:
List(orb.images, PermList);

In [ ]:
PlotGraph(Union(List(orb.images, x -> List([1..10], i-> [i,x[i]]))), rec(directed:= true));

## Exercises

*  Prove the claims made above.  In particular, show that the partition of a (finite) group into conjugacy classes can be computed as an orbit under the right action, provided that the set of generators is closed under conjugation.

* Show that the power set $2^A = \{B : B \subseteq A\}$ of a (finite) set $A$ is the orbit of $A$ under the take-away action:
```gap
takeAway := function(set, s)
    return Difference(set, [s]);
end;
```

* Implement the **membership test** $x \in G$, using stabilizers.

* ($*$) In practice, the number of Schreier generators of the stabilizers in the chain can grow very fast, in larger examples.  The **Schreier-Sims** algorithm intertwines orbit calculations and membership tests to keep the number of necessary generators small.  Implement such a strategy.

* ($**$) Compute the order of the Rubik's cube group
```gap
cube := Group(
( 1, 3, 8, 6)( 2, 5, 7, 4)( 9,33,25,17)(10,34,26,18)(11,35,27,19),
( 9,11,16,14)(10,13,15,12)( 1,17,41,40)( 4,20,44,37)( 6,22,46,35),
(17,19,24,22)(18,21,23,20)( 6,25,43,16)( 7,28,42,13)( 8,30,41,11),
(25,27,32,30)(26,29,31,28)( 3,38,43,19)( 5,36,45,21)( 8,33,48,24),
(33,35,40,38)(34,37,39,36)( 3, 9,46,32)( 2,12,47,29)( 1,14,48,27),
(41,43,48,46)(42,45,47,44)(14,22,30,38)(15,23,31,39)(16,24,32,40) 
);;
```